In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns

In [3]:
file_folder = os.getcwd() 
train_file = file_folder +  '\\data\\yellow_tripdata_2022-01.parquet'
val_file = file_folder +  '\\data\\yellow_tripdata_2022-02.parquet'


In [4]:
jan = pd.read_parquet(train_file)
jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
jan['duration'] = (jan.tpep_dropoff_datetime - jan.tpep_pickup_datetime) / 60

In [7]:
duration_std = np.std(jan.duration)
duration_std

Timedelta('0 days 00:00:46.445295')

In [9]:
duration_sample = jan['duration'].sample(n=500, random_state=42)
duration_sample

1028581   0 days 00:00:08.733333
909034    0 days 00:00:12.616666
1390178   0 days 00:00:14.833333
501663    0 days 00:00:10.150000
2007856   0 days 00:00:04.033333
                   ...          
1210960   0 days 00:00:25.033333
2059859   0 days 00:00:08.366666
890444    0 days 00:00:12.500000
1817967   0 days 00:00:06.600000
96527     0 days 00:00:27.350000
Name: duration, Length: 500, dtype: timedelta64[us]

In [10]:
sns.displot(duration_sample, kde=True)

KeyboardInterrupt: 